In [2]:
import pandas as pd

data = pd.read_csv("data.csv", parse_dates=["Date"], index_col=0)

data = data.interpolate(method="time")

data.head()

,Series1,Series2,Series3,Series4,Series5,Series6
Date,,,,,,
2012-01-01 00:00:00,0.458882,-1.358177,1.893863,1.022005,-1.810638,-0.792524
2012-01-01 00:01:00,0.398794,-1.356111,2.011162,1.104435,-1.807173,-0.726880
2012-01-01 00:02:00,0.480416,-1.354050,2.116713,1.134004,-1.803721,-0.816924
2012-01-01 00:03:00,0.319643,-1.351994,1.987292,0.996250,-1.800283,-1.010524
2012-01-01 00:04:00,0.361104,-1.349943,2.059825,1.041442,-1.796859,-1.041641


In [ ]:
import numpy as np
from scipy import stats

# Calculate z-scores
z_scores = np.abs(stats.zscore(data, nan_policy='omit'))

# Define a threshold for what we consider an outlier
threshold = 3  # typical value: 3 standard deviations

# Replace outliers with NaN
df_no_outliers = data.mask(z_scores > threshold)


data = df_no_outliers.interpolate(method="time")

In [ ]:
data.to_csv("data_clean.csv")